In [1]:
%pwd
%cd ../..

/Users/ryandevera/data-science/umn_environments/Deeplifting


In [2]:
import pandas as pd
import glob

In [3]:
files = glob.glob('./results/algorithm-comparisons-alpine1*')
files

['./results/algorithm-comparisons-alpine1_100d-high-dimensional.parquet',
 './results/algorithm-comparisons-alpine1_3d-high-dimensional.parquet',
 './results/algorithm-comparisons-alpine1_5d-high-dimensional.parquet',
 './results/algorithm-comparisons-alpine1_30d-high-dimensional.parquet']

In [4]:
df = pd.read_parquet(files)

In [5]:
df

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,x97,x98,x99,x100,f,algorithm,time,problem_name,hits,dimensions
0,-0.000582,-0.108272,-1.230785e-02,-0.001517,-5.018434e-09,3.241760,-0.100167,3.241760e+00,9.524945,-3.041425e+00,...,-0.091479,-0.005449,0.004102,-0.063053,1.496774,IPOPT,13.786119,alpine1_100d,0,100
1,-0.007421,0.090246,7.750193e-03,0.008441,-5.002430e-09,-6.383353,-9.324611,-5.001259e-09,-0.580198,-1.077428e-01,...,6.183018,-0.100167,0.007750,3.241760,1.982807,IPOPT,13.545290,alpine1_100d,0,100
2,-0.045073,-0.022780,-4.960424e-09,-0.034211,-5.028158e-09,-0.107638,-9.324611,4.290153e-02,-0.028014,-8.231316e-02,...,3.241760,6.183018,-0.099191,-0.111692,1.361572,IPOPT,13.642626,alpine1_100d,0,100
3,-0.100167,0.008171,-1.001674e-01,-0.044445,1.331356e+00,6.183018,-0.054126,-9.324611e+00,-9.324611,-5.022387e-09,...,3.241760,-3.041425,-0.099541,-9.324611,2.704078,IPOPT,13.866904,alpine1_100d,0,100
4,9.524945,-0.011012,6.183018e+00,-0.105481,-8.903917e-02,-3.041425,-0.128551,1.991570e-05,0.002056,-1.093631e-01,...,-0.087748,-3.041425,3.241760,-0.105909,2.087259,IPOPT,13.706179,alpine1_100d,0,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,...,NaN,NaN,NaN,NaN,0.000000,SCIP,0.045761,alpine1_30d,1,30
96,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,...,NaN,NaN,NaN,NaN,0.000000,SCIP,0.037808,alpine1_30d,1,30
97,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,...,NaN,NaN,NaN,NaN,0.000000,SCIP,0.037086,alpine1_30d,1,30
98,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,...,NaN,NaN,NaN,NaN,0.000000,SCIP,0.037654,alpine1_30d,1,30


In [ ]:
import seaborn as sns

(df.groupby(['problem_name', 'dimensions', 'algorithm'])['hits'].agg('mean'))

In [ ]:
agg_df = (
    df.groupby(['problem_name', 'dimensions', 'algorithm'])['time']
    .agg('mean')
    .reset_index()
)

ax = (agg_df.pivot_table(index='dimensions', columns='algorithm', values='time')).plot(
    figsize=(7, 2.5), cmap='YlOrRd_r'
)
ax.set_title('Ackley Series: Dimensions Vs. Time')
ax.set_xlabel('Dimensions (N)')
ax.set_ylabel('Time (Seconds)')
ax.grid()

markers = ['o', 'x', 's', 'D', '^', 'v', '<', '>', 'p', '*']

for i, line in enumerate(ax.get_lines()):
    line.set_marker(markers[i])

ax.legend()

In [ ]:
agg_df

In [ ]:
import torch
import numpy as np
from deeplifting.problems import HIGH_DIMENSIONAL_PROBLEMS_BY_NAME

problem = HIGH_DIMENSIONAL_PROBLEMS_BY_NAME['ackley_2500d']

In [ ]:
x = torch.zeros(2500)
results = np.zeros((1, 1, 2500 + 1))
trial = 0

In [ ]:
%%timeit
f = problem['objective']
f(x, results=results, trial=trial, version='pytorch')

In [ ]:
%%timeit
multi_result = [x] * 512
f_list = []
for xr in multi_result:
    r = f(xr, results=results, trial=trial, version='pytorch')
    f_list.append(r)

# Figure out performance

In [ ]:
import time

from pygranso.private.getNvar import getNvarTorch
from pygranso.pygranso import pygranso
from pygranso.pygransoStruct import pygransoStruct
from deeplifting.optimization import deeplifting_nd_fn, deeplifting_predictions
from deeplifting.utils import HaltLog

# first party
from deeplifting.models import DeepliftingSkipMLP

In [ ]:
def ndschwefel(x, version='pytorch'):
    """
    Implemention of the n-dimensional levy function

    Args:
    x: A d-dimensional array or tensor
    version: A string, either 'numpy' or 'pytorch'

    Returns:
    result: Value of the Rastrigin function
    """
    x = x.flatten()
    d = len(x)
    if version == 'numpy':
        result = 418.9829 * d - np.sum(x * np.sin(np.sqrt(np.abs(x))))
    elif version == 'pytorch':
        result = 418.9829 * d - torch.sum(x * torch.sin(torch.sqrt(torch.abs(x))))
    else:
        raise ValueError(
            "Unknown version specified. Available options are 'numpy' and 'pytorch'."
        )

    return result

In [ ]:
%%timeit
bounds = [(-32, 32)] * 2500

In [ ]:
# Deeplifting model with skip connections
model = DeepliftingSkipMLP(
    input_size=512,
    hidden_sizes=(512, 512, 512),
    output_size=2500,
    bounds=bounds,
    skip_every_n=1,
    activation='sine',
    output_activation='leaky_relu',
    agg_function='sum',
    seed=0,
)

In [ ]:
%%time
outputs = model(None)

In [ ]:
%%time
ndschwefel(outputs[0, :], version='pytorch')

In [ ]:
import tqdm


def deeplifting_nd_fn(model, objective, method='particle'):
    """
    Combined funtion used for PyGranso
    """
    outputs = model(inputs=None)

    # Get x1 and x2 so we can add the bounds
    # outputs = torch.sigmoid(outputs)
    # x = outputs.mean(axis=0)
    # print(f'Output x {x}')
    x, f = deeplifting_predictions(outputs, objective, method=method)

    return f

In [ ]:
%%time
fn = lambda x: objective(x, version='pytorch')
deeplifting_nd_fn(model, fn)

In [ ]:
device = torch.device('cpu')
dimensions = 2500
trials = 1
method = 'particle'
objective = ndschwefel

model = model.to(device=device, dtype=torch.double)
nvar = getNvarTorch(model.parameters())

# Setup a pygransoStruct for the algorithm
# options
opts = pygransoStruct()

# Inital x0
x0 = (
    torch.nn.utils.parameters_to_vector(model.parameters())
    .detach()
    .reshape(nvar, 1)
    .to(device=device, dtype=torch.double)
)

opts.x0 = x0
opts.torch_device = device
opts.print_frequency = 1
opts.limited_mem_size = 5
opts.stat_l2_model = False
opts.double_precision = True
opts.opt_tol = 1e-5
opts.maxit = 2

# results

# Set up the function with the results
fn = lambda x: objective(x, version='pytorch')  # noqa

# # Combined function
comb_fn = lambda model: deeplifting_nd_fn(
    model,
    fn,
    method=method,
)  # noqa

# Run the main algorithm
start_time = time.time()
soln = pygranso(var_spec=model, combined_fn=comb_fn, user_opts=opts)
end_time = time.time()
total_time = end_time - start_time

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import tqdm


def deeplifting_nd_fn(model, objective, method='particle'):
    """
    Combined funtion used for PyGranso
    """
    outputs = model(inputs=None)

    # Get x1 and x2 so we can add the bounds
    # outputs = torch.sigmoid(outputs)
    f = deeplifting_predictions(outputs, objective, method=method)

    return f


def deeplifting_predictions(x, objective, method='particle'):
    """
    Convert scaled values to the objective function
    """
    if method == 'particle':
        # Iterate over the objective values
        objective_values = []
        for i in range(len(x)):
            f = objective(x[i, :])
            objective_values.append(f)

        objective_values = torch.stack(objective_values)
        f = torch.min(objective_values)

    return f


bounds = [(-500, 500)] * 2
device = torch.device('cpu')

# Initialize model
model = DeepliftingSkipMLP(
    input_size=512,
    hidden_sizes=(1024, 1024, 1024),
    output_size=2,
    bounds=bounds,
    skip_every_n=1,
    activation='sine',
    output_activation='leaky_relu',
    agg_function='sum',
    seed=0,
)
model = model.to(device=device, dtype=torch.double)
model.train()

objective = ndschwefel

# Define the optimizer
# optimizer = optim.LBFGS(model.parameters(), lr=1e-3, line_search_fn='strong_wolfe')
optimizer = optim.Adam(model.parameters(), lr=1e-5)

criterion = lambda m: deeplifting_nd_fn(m, objective)

# Training loop
for epoch in range(250):
    # def closure():
    #     # Zero out the gradients
    #     optimizer.zero_grad()

    #     # The loss is the sum of the compliance
    #     loss = criterion(model)

    #     # Go through the backward pass and create the gradients
    #     loss.backward()

    #     return loss

    # # Step through the optimzer to update the data with the gradients
    # optimizer.step(closure)
    optimizer.zero_grad()

    loss = criterion(model)
    loss.backward()

    optimizer.step()

    print(criterion(model))

In [ ]:
%%timeit
criterion(model)

In [ ]:
%%time
criterion(model)

# Layeb 5

In [ ]:
import numpy as np

x = np.array([-np.pi, 0.0] * 1000)

xi = x[:-1]
xj = x[1:]

component1 = np.log(np.abs(np.sin(xi - np.pi / 2) + np.cos(xj - np.pi)) + 1e-3)
component2 = np.abs(np.cos(2 * xi - xj + np.pi / 2)) + 1
np.sum(component1 / component2)

In [ ]:
np.log(1e-3) * (len(x) - 1)

# Layeb 8

In [ ]:
x = np.array([np.pi / 4, -np.pi / 4] * 200)

xi = x[:-1]
xj = x[1:]

component1 = np.log(np.abs(xi + xj) + 1e-3)
component2 = np.abs(100 * np.cos(xi - xj))
# print(component1)

np.sum(component1 + component2)

In [ ]:
np.log(1e-3) * (len(x) - 1)

# Layeb 11

In [ ]:
x = np.array([0, -1.0] * 500)

xi = x[:-1]
xj = x[1:]

component1 = np.cos(xi * xj + np.pi)
component2 = (100 * np.abs(xi**2 - xj - 1.0)) ** 2 + 1.0

np.sum(component1 / component2)

In [ ]:
-1.0 * len(x) + 1

# Layeb 12

In [ ]:
x = np.array([2.0] * 1000)

xi = x[:-1]
xj = x[1:]

component1 = np.cos(np.pi / 2.0 * xi - np.pi / 4.0 * xj - np.pi / 2.0)
component2 = np.exp(np.cos(16.0 * np.pi * xi * xj))

-np.sum(component1 * component2 + 1.0)

In [ ]:
-(np.e + 1) * (len(x) - 1)